## Import 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install graphviz;
#!pip uninstall dtreeviz

In [ ]:
## look into the dater
#from fastbook import *
#from kaggle import api
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from fastai.tabular.all import *
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor
#!pip install dtreeviz
#from dtreeviz.trees import *
from IPython.display import Image, display_svg, SVG

In [ ]:
from sklearn.metrics import confusion_matrix, roc_auc_score, plot_confusion_matrix, precision_score, recall_score, classification_report, plot_roc_curve, roc_curve
from sklearn.model_selection import train_test_split

In [ ]:
## pandas
pd.options.display.max_rows = 20
pd.options.display.max_columns = 12

In [ ]:
main_path = "../input/ieee-fraud-detection-joined-tables/"

### Random Sampling

In [ ]:
## 1. Create df from 100k sample and then go to random 50k sample
df = pd.read_csv(main_path+"tr_sample-100k.csv")
df = df.sample(50000, random_state=42)
# #df_te= pd.read_csv(main_path + "te_tran_iden.csv")

### Undersampling

In [ ]:
# ## 2. Take all samples of isFraud=True and Random Samples of isFraud=False
# df = pd.read_csv(main_path+"tr_tran_iden.csv")
# ### ratio of true and false
# print(df.isFraud.value_counts()[1]/len(df.isFraud) *100)
# print(df["isFraud"].value_counts())

In [ ]:
# df1 = df[df["isFraud"]==False].sample(50000, random_state=42)
# df2 = df[df["isFraud"]==True]
# df = pd.concat([df1,df2], ignore_index=True)
# df.shape, df1.shape, df2.shape

In [ ]:
# ### ratio of true and false
# df.isFraud.value_counts()[1]/len(df.isFraud) *100

## oversampling (with replacement)

In [ ]:
# ## Get dataset
# df = pd.read_csv(main_path+"tr_sample-100k.csv")
# ### ratio of true and false
# df = df.sample(50000,random_state=42)
# df.isFraud.value_counts()


In [ ]:
# ## sort by time
# df.sort_values("TransactionDT", inplace=True)
# df.reset_index(drop=True, inplace=True)
# df

In [ ]:
# ## split the data into train and valid
# df1 = df.iloc[0:int(0.8*len(df)),]
# df2 = df.iloc[int(0.8*len(df)):len(df),]
# df1.shape,df2.shape

In [ ]:
# ## Mix both the ROSed train dater and valid dater
# from imblearn.over_sampling import RandomOverSampler
# ros = RandomOverSampler(random_state=42)

# X,Y = ros.fit_resample(df1,df1["isFraud"])
# df=pd.concat([X,df2], ignore_index=True)
# df.reset_index(drop=True, inplace=True)

In [ ]:
#len(X)/len(df), df.isFraud.value_counts(), df2.isFraud.value_counts()

## Check Dater 

In [ ]:
## check tables for missing values in join column
col = "shape", "columns with nans ", "nans in TransactioniD"
ind = "train","test"
lst1 = [df.shape, sum(df.isna().sum()>0), df.TransactionID.isna().sum()]
#lst2 = [df_te.shape, sum(df_te.isna().sum()>0), df_te.TransactionID.isna().sum()]
#pd.DataFrame([lst1,lst2], columns=col,index=ind)
lst1

In [ ]:
## Class imbalance? Might need to upsample or downsample???? Isn't usage of false positves and true positives overcoming imbalance?
sum(df["isFraud"]==True)/len(df)

## Trimming col from the dater

In [ ]:
## remove columns with nans>300k
##[((df.isna().sum()>n/2*1e5).sum(),n/2) for n in range(1,13)]
#to_drop = df.columns[df.isna().sum()>300000].to_list()
to_drop=[] # forcing to use all variables
df.drop(to_drop,1, inplace=True)
df.shape

## Tabular Pandas

In [ ]:
## sort by time
df.sort_values("TransactionDT", inplace=True)
df.reset_index(drop=True, inplace=True)
df

In [ ]:
## split to train and valid ind linearly
msk = np.arange(len(df))<0.8*len(df) #usually 0.8, 0.8851 for the oversampling case
splits = (list(np.where(msk)[0]),list(np.where(~msk)[0]))
len(splits[0])+len(splits[1]), len(df), len(splits[0])

In [ ]:
## variables for TP
dep_var = "isFraud"
procs = [Categorify,FillMissing]
## Cont and Cat variables
cont,cat = cont_cat_split(df, 1, dep_var=dep_var)

In [ ]:
## check if cont variables in test have nans where train doesn't
#def na_check(col): return df.loc[:,col].isna().sum()>0,df_te.loc[:,col].isna().sum()>0
#na_col = {col: na_check(col) for col in cont}
#na_col_te = [col for col in cont if df.loc[:,col].isna().sum()==0 and df_te.loc[:,col].isna().sum()>0]
na_col_te = ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14']


In [ ]:
df.isFraud.value_counts()

In [ ]:
## Add a last row with nans for col:na_col_te
#df.insert(0,df.columns,df.median())

df = pd.concat([df.median().to_frame().T, df], ignore_index=True)

In [ ]:
df.loc[df.index[0],na_col_te] = np.nan
df.loc[df.index[0],"isFraud"]=1 ## forcing it to be an int rather than in between like 0.5
df.loc[df.index[0],na_col_te]
df.shape

In [ ]:
df.isFraud.value_counts()

In [ ]:
to = TabularPandas(df, procs, cat, cont, y_names=dep_var, splits=splits)
len(to.train)+len(to.valid)

In [ ]:
to.items.columns

In [ ]:
to.valid.show(3)

In [ ]:
temp = to.valid.items
temp.isFraud.value_counts()

In [ ]:
to.items.head()

In [ ]:
##vocab
to.classes["ProductCD"]

In [ ]:
save_pickle("to.pkl",to)

In [ ]:
load_pickle("to.pkl")

## Model

In [ ]:
def rf(xs, y, n_estimators=40, max_samples=0.6,#20_000,#200_000,
       max_features=0.8, min_samples_leaf=5, sample_weight=None, **kwargs):
    return RandomForestClassifier(n_jobs=-1, n_estimators=n_estimators,
        max_samples=max_samples, max_features=max_features,
        min_samples_leaf=min_samples_leaf, oob_score=True,random_state=42).fit(xs, y,sample_weight)#, class_weight="balanced_subsample").fit(xs, y)

In [ ]:
xs,y = to.train.xs,to.train.y
valid_xs,valid_y = to.valid.xs,to.valid.y
xs.shape

In [ ]:
# ## Oversampling SMOTE
# from imblearn.over_sampling import SMOTE
# sm = SMOTE(random_state=42, n_jobs=-1,k_neighbors=15)

# X,Y = sm.fit_resample(xs,y)
# m = rf(X,Y)
#Y.value_counts(), y.value_counts(),

In [ ]:
y.dtype

In [ ]:
# #oversampling
# from imblearn.over_sampling import RandomOverSampler
# ros = RandomOverSampler(random_state=42)

# X,Y = ros.fit_resample(xs,y)
# m = rf(X,Y)

In [ ]:
## Weighting the samples
# wt = y.value_counts()[0]/y.value_counts()[1]
# y.value_counts(),wt
#sample_weight = np.array([0.88 if i == 0 else 23 for i in y])

In [ ]:
## m = DecisionTreeRegressor(max_leaf_nodes=4)
## m.fit(xs, y);
m = rf(xs,y)#,sample_weight=sample_weight)
#Y=y


In [ ]:
def r_mse(pred,y): return round(math.sqrt(((pred-y)**2).mean()), 6)

In [ ]:
m1 = m.estimators_[0]

In [ ]:
## check the leaves are much lesser than the length of samples.
max([t.get_n_leaves() for t in m.estimators_]), m.max_samples*len(xs)

## Feature importance

In [ ]:
def rf_feat_importance(m, df):
    return pd.DataFrame({'cols':df.columns, 'imp':m.feature_importances_}
                       ).sort_values('imp', ascending=False)

In [ ]:
pd.DataFrame(m.feature_importances_).describe()

In [ ]:
fi = rf_feat_importance(m,xs)
fi[0:20]

In [ ]:
def plot_fi(fi):
    return fi.plot('cols', 'imp', 'barh', figsize=(12,7), legend=False)

plot_fi(fi[:50]);

In [ ]:
to_keep = fi[fi.imp>0.005].cols
len(to_keep)

In [ ]:
fi.imp

In [ ]:
xs_cp = xs.copy()
valid_xs.cp = valid_xs.copy()

In [ ]:
xs= xs[to_keep]
valid_xs = valid_xs[to_keep]

In [ ]:
xs_imp.shape

In [ ]:
m=rf(xs,y)

## RF metrics deep dive

In [ ]:
## checking if mean of all estimators gives the random forest results
preds_p = np.stack([t.predict_proba(valid_xs)[:,1] for t in m.estimators_])
r_mse(m.predict_proba(valid_xs)[:,1],preds_p.mean(0))


In [ ]:
## how AUC score changes with n_estimators_
plt.plot([roc_auc_score(valid_y, preds_p[:i+1].mean(0)) for i in range(len(m.estimators_))]); # n_estimators seems fine

In [ ]:
m.max_samples

## OUT OF BAGs

In [ ]:
## predict proba of OOB
y_oob = m.oob_decision_function_[:,1]
y_oob.shape

In [ ]:
oob_auc = roc_auc_score(y,m.oob_decision_function_[:,1])

In [ ]:
fpr,tpr,_ = roc_curve(y,y_oob)
plt.plot(fpr,tpr, label='AUC = ' + str(round(roc_auc_score(y,m.oob_decision_function_[:,1]), 2)))
plt.legend(loc='lower right')

In [ ]:
roc_auc_score(y,y_oob)

## STD across trees gives confidence in the readings

In [ ]:
preds_p.shape

In [ ]:
preds_p_std = preds_p.std(0)
preds_p_std[0:5]

In [ ]:
pd.DataFrame(preds_p_std).describe() ## Can look at individual rows prediction std from different trees.

## Output of Valid dater

In [ ]:
## how unbalanced is the train and the valid, they look the same to me. :)
y.value_counts()[1]/y.value_counts()[0] *100, valid_y.value_counts()[1]/valid_y.value_counts()[0] *100

In [ ]:
## Metrics functions
def m_rmse(m, xs, y): return r_mse(m.predict(xs), y)
#def m_accuracy(m,xs,y): return sum(m.predict(xs)==y)/len(xs)
def m_accuracy2(m,xs,y): return m.score(xs,y)
def m_conf(m,xs,y): 
    plot_confusion_matrix(m,xs,y)
    return confusion_matrix(y, m.predict(xs))
def m_prec(m,xs,y): return precision_score(y,m.predict(xs), average=None, labels=[1,0])
def m_recall(m,xs,y): return recall_score(y,m.predict(xs), average=None, labels=[1,0])
def m_rep(m,xs,y): return classification_report(y, m.predict(xs), labels=[1,0], digits=4, output_dict=True)

In [ ]:
## See predicted column. Does it match expectations
pd.DataFrame(m.predict(xs)).describe() 

In [ ]:
## prec recal train
tr_rep = m_rep(m,xs,y)
print(tr_rep["1"]["recall"]) ## sensitivity TPR
print(tr_rep["0"]["recall"]) ## TNR
print(1- tr_rep["0"]["recall"]) ## 1-specificity FPR
print(tr_rep["1"]["precision"])
print(tr_rep["0"]["precision"])

In [ ]:
## prec recal valid
vd_rep = m_rep(m,valid_xs,valid_y)
print(vd_rep["1"]["recall"]) ## sensitivity TPR
print(vd_rep["0"]["recall"]) ## TNR
print(1- vd_rep["0"]["recall"]) ## 1-specificity FPR
print(vd_rep["1"]["precision"])
print(vd_rep["0"]["precision"])

In [ ]:
# plot_roc_curve(m,xs,y,response_method="decision_function") not working 
plt.plot(fpr,tpr, label='AUC = ' + str(round(roc_auc_score(y,m.oob_decision_function_[:,1]), 2)))
plt.legend(loc='lower right')


In [ ]:
## plot ROC
plot_roc_curve(m,xs,y)
plot_roc_curve(m,valid_xs,valid_y)

In [ ]:
## AUC score
print(roc_auc_score(y,m.predict_proba(xs)[:,1]), roc_auc_score(valid_y,m.predict_proba(valid_xs)[:,1]))
## recall
print("\n",tr_rep["1"]["recall"]) ## sensitivity TPR
print(tr_rep["0"]["recall"]) ## TNR
##valid
print(vd_rep["1"]["recall"]) ## sensitivity TPR
print(vd_rep["0"]["recall"]) ## TNR
## precision
print(tr_rep["1"]["precision"])
print(tr_rep["0"]["precision"])
## valid precision
print(vd_rep["1"]["precision"])
print(vd_rep["0"]["precision"])

In [ ]:
print(round(roc_auc_score(y,m.predict_proba(xs)[:,1]),4),";",round(roc_auc_score(y,m.oob_decision_function_[:,1]),4),";",round(roc_auc_score(valid_y,m.predict_proba(valid_xs)[:,1]),4),";",
      round(tr_rep["1"]["recall"],4),",", round(tr_rep["0"]["recall"],4),";",
      round(vd_rep["1"]["recall"],4), ",",round(vd_rep["0"]["recall"],4),";",
      round(tr_rep["1"]["precision"],4),",",round(tr_rep["0"]["precision"],4),";",
      round(vd_rep["1"]["precision"],4),",",round(vd_rep["0"]["precision"],4))

In [ ]:
precision_score(y,m.predict(xs))

## Resources
More info on the dater
1. https://www.kaggle.com/c/ieee-fraud-detection/discussion/101203 
1. [Notebook on RF](https://www.kaggle.com/raviolli77/random-forest-in-python#Training-Algorithm)
2. [searching hyperparameters](https://www.kaggle.com/raviolli77/random-forest-in-python#Creating-Training-and-Test-Sets)
3. [8 tactics to combat unbalanced datersets](https://machinelearningmastery.com/tactics-to-combat-imbalanced-classes-in-your-machine-learning-dataset/)
4. [Kaggle notebook on unbalanced datersets](https://www.kaggle.com/janiobachmann/credit-fraud-dealing-with-imbalanced-datasets)
5. [Survey of predictive modeling with imbalnced datersets](https://arxiv.org/abs/1505.01658)